# Exercise 2.1 - Compilation modes

## Objective

- Understand the differences in typing between nopython mode and object mode.
- See how to force nopython mode compilation
- Understand what typing errors due to failed unification look like
- Understand what typing errors due to unsupported functions look like

## Object and Nopython mode

Define a function which is compiled by Numba, and the same function again, with the restriction that it must be compiled on nopython mode:

In [ ]:
from numba import jit

@jit
def add(a, b):
    return a + b

@jit(nopython=True)
def add_nopython(a, b):
    return a + b

We can call add with unsupported types:

In [ ]:
add("A", "B")

Have a look at the typing which Numba generated:

In [ ]:
add.inspect_types()

The `pyobject` type is any Python type - that is, Numba does not know or cannot constrain the types of a variable with this type. As a result, the generated code contains calls to CPython C API functions:

In [ ]:
def show_asm(func):
    asm_code = func.inspect_asm()
    for k, v in asm_code.items():
        print('-' * 80)
        print("Signature:", k)
        print('-' * 80)
        print(v)
        print()

show_asm(add)

Notice how there are calls to functions such as `Py_IncRef`, etc. - the need to involve the Python interpreter and reference counting can create a lot of additional overhead.

Using nopython mode avoids these overheads, by making any operation that would require help from the Python interpreter an error. Try:

In [ ]:
add_nopython("A", "B")

Strings are not supported in nopython mode, resulting in the error. The add_nopython function does work for supported types:

In [ ]:
add_nopython(1, 2)

## Unification

Not only must all types be resolved to a supported type when using nopython mode, all types must also unify to a single supported type at the time of compilation. For example:

In [ ]:
@jit(nopython=True)
def select(a, b, c):
    if c:
        x = a
    else:
        x = b
    return x

select(1, (1, 2), True)

Note that the typing cannot depend on the runtime behaviour of the code. In this example, it may be that select is only ever called with `c = True`, so the function would only ever return one type. However, Numba cannot know this, and must always generate typing and code for all paths.

## Unsupported functions

Finally, let’s try calling a function that is not supported in nopython mode:

In [ ]:
import numpy as np

@jit(nopython=True)
def scaled_det(A, x):
    return np.linalg.det(A) * x

scaled_det(np.arange(9).reshape(3,3), 2.0)

The above error message indicates that the `det` method is unsupported by Numba.

## Summary

- You can instruct Numba to only use nopython mode by passing `nopython=True` to the jit decorator.
- This doesn't change the way Numba works, but makes it an error if the function can't be compiled in nopython mode.
- Type unification is done without any analysis to determine whether a branch will be taken or not taken - all branches must be able to be unified.
- Calling functions unsupported by Numba will result in an error in nopython mode.